In [26]:
import torch
from torch.utils.tensorboard import SummaryWriter

In [27]:
import os

def delete_folder(folder_path):
    if os.path.exists(folder_path):
        for root, dirs, files in os.walk(folder_path, topdown=False):
            for name in files:
                os.remove(os.path.join(root, name))
            for name in dirs:
                os.rmdir(os.path.join(root, name))
        os.rmdir(folder_path)
    else:
        print(f"The folder {folder_path} does not exist")

delete_folder('./runs')

In [28]:
'''
Pre-defined constants
'''
n = 55000  # The size of training set
scale = 1e-3 # the scale factor for loss function
damp = 1e-2 # the damp factor to add L2 regularization in loss function
criterion = torch.nn.CrossEntropyLoss() # loss function without scaling and regularization

In the section below, we would load training dataset and testing dataset from MNIST. We assign the training set size $n=55000$, which is accorded to the paper Understanding Black-box Predictions via Influence Functions

In [29]:
from torchvision import datasets, transforms
from torch.utils.data import Subset

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Example: Normalize to mean=0.5, std=0.5
])

train_dataset_all = datasets.MNIST(root='./data', train=True, download=True,
                               transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True,
                              transform=transform, )

train_indices = torch.randperm(len(train_dataset_all))[:n]
train_dataset = Subset(train_dataset_all, train_indices)

In [30]:
all_data_list = []
all_labels_list = []
for i in range(len(train_dataset)):
    data, label = train_dataset[i]
    all_data_list.append(data)         # img shape: [1, 28, 28]
    all_labels_list.append(label)

train_data = torch.stack(all_data_list, dim=0)  # shape [n, 1, 28, 28]
train_labels = torch.tensor(all_labels_list)        # shape [n]

## MNIST Logistic Regression Model Definition

Below we defined the logistic regression model with training and testing method using L-BFGS optimizer

In [31]:
import torch.nn as nn

class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim, params=None):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        if params:
            self.linear.weight = nn.Parameter(params['weight'])
            self.linear.bias = nn.Parameter(params['bias'])
        
    def forward(self, x):
        x = x.view(-1, self.linear.in_features)
        # outputs = torch.nn.functional.softmax(self.linear(x))
        outputs = self.linear(x)
        return outputs

In [32]:
def train_lbfgs(model, criterion, train_data, train_labels, writer=None, t=20, leave_out_index=None):
    optimizer = torch.optim.LBFGS(model.parameters(), lr=0.1, max_iter=50, line_search_fn='strong_wolfe')
    # If there is a leave-out indices list, exclude these indies from the training set
    if leave_out_index is not None:
        train_data = torch.cat((train_data[:leave_out_index], train_data[leave_out_index + 1:]), dim=0) 
        train_labels = torch.cat((train_labels[:leave_out_index], train_labels[leave_out_index + 1:]), dim=0)        
    
    for epoch in range(t):  
        def closure():
            optimizer.zero_grad()
            output = model(train_data)
            loss = criterion(output, train_labels)
            loss.backward()
            return loss
        
        loss_val = optimizer.step(closure)
        if writer:
            writer.add_scalar('training loss', loss_val, epoch)
        # print(f"epoch {epoch} finished, loss={loss_val}")        
    return

In [33]:
'''
Batch validation for the model
'''
def test(model, criterion, test_dataset):
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

    val_loss = 0
    val_correct = 0
    incorrect_data_list, incorrect_label_list = [], []
    with torch.no_grad():  # Disable gradient calculation during validation
        for data, labels in test_loader:
            # Forward pass
            outputs = model(data.view(-1, 28*28))

            # Calculate loss
            loss = criterion(outputs, labels)

            # Update validation metrics (e.g., accuracy)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_correct += (predicted == labels).sum().item()
            incorrect_data_list.append(data[predicted != labels])
            incorrect_label_list.append(labels[predicted != labels])
            
    incorrect_data = torch.cat(incorrect_data_list, dim=0)
    incorrect_label = torch.cat(incorrect_label_list, dim=0)
    val_loss /= len(test_dataset)
    val_accuracy = 100 * val_correct / len(test_dataset)
    return val_loss, val_accuracy, incorrect_data, incorrect_label

## Train the baseline model in L-BFGS Optimizer and calculate $L(z, \hat{\theta})$

In [34]:
'''
Create a summary writer for plotting training loss
'''
model_train_writer = SummaryWriter('runs/logistic_regression_10_mnist_train')

In [35]:
base_model = LogisticRegression(28*28, 10)
# Store the initial parameter values for later leave-one-out retrain
initial_params = {"weight": base_model.linear.weight.data.clone(), "bias": base_model.linear.bias.data.clone()}

def criterion_l2(output, target, model):
    loss = criterion(output, target) * scale
    l2_reg = 0.0
    for param in model.parameters():
        l2_reg += torch.norm(param, 2)**2
    loss += damp * l2_reg
    return loss

base_model_criterion = lambda x, y: criterion_l2(x, y, base_model)
train_lbfgs(base_model, base_model_criterion, train_data, train_labels, writer=model_train_writer, t=10)

In [36]:
val_loss, val_accuracy, incorrect_data, incorrect_label = test(base_model, base_model_criterion, test_dataset)
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

Validation Loss: 0.0000, Validation Accuracy: 62.23%


## Calculate loss difference of leave-one-out retraining

According to the paper, we arbitrarily picked a wrongly-classified test point $z_{test}=(x_{test}, y_{test})$ as below

In [37]:
test_data_index = torch.randint(0, len(incorrect_data), (1,)).item()
x_test, y_test = incorrect_data[test_data_index], incorrect_label[test_data_index].view(1)

In [38]:
def calc_leave_out_loss_diff(model, initial_params, criterion_l2, test_data, test_label, leave_out_index=None, t=1):
    # Clone initial parameters
    params = {"weight": initial_params["weight"].clone(), "bias": initial_params["bias"].clone()}
    
    # Calculate L(z_{test}, \theta) which model is trained with all training points
    loss_z_test_with_z = criterion_l2(model(test_data), test_label, model)
    
    # Train leave-one-out model
    retrained_model = LogisticRegression(28*28, 10, params=params)
    retrained_model_criterion = lambda x, y: criterion_l2(x, y, retrained_model)
    train_lbfgs(retrained_model, retrained_model_criterion, train_data, train_labels, t=t, leave_out_index=leave_out_index)
    
    # Calculate L(z_{test}, \theta_{-z}) which model is trained without data point z
    loss_z_test_without_z = retrained_model_criterion(retrained_model(test_data), test_label)
    
    leave_out_loss_diff = loss_z_test_without_z - loss_z_test_with_z
    print(f"loss_z_test_with_z: {loss_z_test_with_z}, loss_z_test_without_z: {loss_z_test_without_z}, leave_out_loss_diff: {leave_out_loss_diff}")
    return leave_out_loss_diff

In [39]:
'''
Validate the functionality of calc_leave_out_loss_diff
'''
leave_out_loss_diff = calc_leave_out_loss_diff(base_model, initial_params, criterion_l2, x_test, y_test, t=10, leave_out_index=None)
assert leave_out_loss_diff == 0

loss_z_test_with_z: 0.002304940717294812, loss_z_test_without_z: 0.002304940717294812, leave_out_loss_diff: 0.0


## Implement Influence Function

### 0. Preparation

Since in influence function stochastic estimation, we need to uniformly sample $t$ points from training set, an uniform sampling method would be neccessary to be defined as below:

In [40]:
def uniform_sample_dataset(dataset, t, batch_size=1):
    sampler = torch.utils.data.sampler.RandomSampler(dataset, num_samples=t)
    sampled_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=sampler)
    return sampled_dataloader

Define a method to calculate first order gradient of multivariable scalar function (in our case, loss function) for all the parameters:
$$\nabla_{\theta} L(z, \theta)$$

In [41]:
def calc_first_order_gradient(loss, model):
    model.zero_grad()
    # Compute the first-order gradient
    loss.backward(create_graph=True)
    param_grads = [ p.grad.flatten() for p in model.parameters() if p.requires_grad ]
    param_grads = torch.cat(param_grads)
    return param_grads

Define a method to calculate second order gradient of multivariable scalar function (i.e. Hessian matrix) for all the parameters:
$$\nabla^{2}_{\theta} L(z, \theta)$$

Noted this method would not be used for any reason other than validation purpose, since it is not efficient and for our implementation, HVP (Hessian Vector Product) would be a better choice.

In [42]:
def calc_second_order_gradient(loss, model):
    model.zero_grad()
    # Compute the first-order gradient
    loss.backward(create_graph=True)
    first_grads = [ p.grad.flatten() for p in model.parameters() if p.requires_grad ]
    # Compute the second-order gradient
    second_grads = []
    for first_grad in first_grads:
        row = []
        for p in model.parameters():
            # Compute every possible combination of differentiation variables for the parameters
            sub_matrix = []
            for i in range(first_grad.shape[0]):
                sub_matrix.append(torch.autograd.grad(first_grad[i], p, create_graph=True)[0].flatten())
            sub_matrix = torch.stack(sub_matrix)
            row.append(sub_matrix)
        row = torch.cat(row, dim=1)
        second_grads.append(row)
    second_grads = torch.cat(second_grads, dim=0)
    return second_grads

### 1. HVP (Hessian Vector Product) Calculation

This method could accer

In [43]:
def hvp(loss, params, vector):
    # Compute the first-order gradient
    first_grads = torch.autograd.grad(loss, params, create_graph=True)

    first_grads = [g.flatten() for g in first_grads]
    first_grads = torch.cat(first_grads, dim=0)
    
    # Calculate the product between the first gradients and the vector
    first_grad_vector_product = torch.sum(first_grads * vector)

    # Compute the second-order gradient
    second_grads = torch.autograd.grad(first_grad_vector_product, params, create_graph=True)
    second_grads = [g.flatten() for g in second_grads]
    second_grads = torch.cat(second_grads, dim=0)

    return second_grads

**Validation for HVP**

In [44]:
'''
Validate the correctness of HVP's implementation
'''

test_model = LogisticRegression(2, 2)
test_criterion = torch.nn.MSELoss()

data_tensor = torch.tensor([[1.0, -2.0]], requires_grad=True)
label_tensor = torch.tensor([1.0, 0.0])

params = [p for p in test_model.parameters()]
vector = calc_first_order_gradient(test_criterion(test_model(data_tensor), label_tensor), test_model)
expected = torch.matmul(calc_second_order_gradient(test_criterion(test_model(data_tensor), label_tensor), test_model), vector.T)
vector._grad_fn = None
test_model.zero_grad()
actual = hvp(torch.nn.MSELoss()(test_model(data_tensor), label_tensor), params, vector)
print(f"Expected HVP: {expected} \nActual HVP: {actual} \nDiff Sum: {(expected-actual).sum()}")
assert torch.equal(expected, actual)

Expected HVP: tensor([ 0.5072, -1.0144, -5.7246, 11.4493,  0.5072, -5.7246],
       grad_fn=<MvBackward0>) 
Actual HVP: tensor([ 0.5072, -1.0144, -5.7246, 11.4493,  0.5072, -5.7246],
       grad_fn=<CatBackward0>) 
Diff Sum: 0.0


### 2. IHVP (Inverse Hessian Vector Product) Calculation



In [45]:
def ihvp(train_dataset, test_data, test_label, model, criterion, t, r, unique_datapoint=None, ihvp_summary_writer=None):
    ihvp_eval_avg = 0
    vector = calc_first_order_gradient(criterion(model(test_data), test_label), model)
    for i in range(r):
        # Arg unique_datapoint is used for debugging. It would generate an assigned single-data dataset
        if unique_datapoint is None:
            sampled_train_dataset = [(data, label) for data, label in uniform_sample_dataset(train_dataset, t)]
        else:
            sampled_train_dataset = []
            for _ in range(t):
                sampled_train_dataset.append(unique_datapoint)
        # Step 1. Initialize the evaluation of the Hessian-vector product
        ihvp_eval = vector
        data_number = 0
        for data, label in sampled_train_dataset:
            # Step 2. Compute the second order gradient of the loss w.r.t. the model parameters
            model.zero_grad()
            data_tensor = data.view(-1, 28*28)
            params = [p for p in model.parameters()]
            ihvp_eval._grad_fn = None
            # Step 3. Compute the inner product between the Hessian matrix and the test gradient vector using HVP
            second_grads = hvp(criterion(model(data_tensor), label), params, ihvp_eval)
            ihvp_eval = ihvp_eval + vector - second_grads
            if ihvp_summary_writer:
                ihvp_summary_writer.add_scalar(f'ihvp_eval_sum_{i}', ihvp_eval.sum(), data_number)
            data_number += 1
        print(f"ihvp iteration {i} done and ihvp sum is {ihvp_eval.sum()}")
        ihvp_eval_avg = i / (i + 1) * ihvp_eval_avg + 1 / (i + 1) * ihvp_eval         
    return ihvp_eval_avg

**Validation for IHVP**: 
Check if IHVP successfully convergent

In [46]:
ihvp_summary_writer = SummaryWriter('runs/ihvp_sum_summary') 
print(f"Test data value sum: {x_test.sum()}, Test data label: {y_test}")
ihvp_eval = ihvp(train_dataset, x_test, y_test, base_model, base_model_criterion, 5000, 5, ihvp_summary_writer=ihvp_summary_writer)
print(ihvp_eval)

Test data value sum: -452.02349853515625, Test data label: tensor([2])
ihvp iteration 0 done and ihvp sum is 0.0005269250832498074
ihvp iteration 1 done and ihvp sum is 0.000527174212038517
ihvp iteration 2 done and ihvp sum is 0.0005267518572509289
ihvp iteration 3 done and ihvp sum is 0.0005266759544610977
ihvp iteration 4 done and ihvp sum is 0.0005274969153106213
tensor([-0.0021, -0.0021, -0.0021,  ...,  0.0015,  0.0021,  0.0014],
       grad_fn=<AddBackward0>)


In [47]:
ihvp_eval.sum()

tensor(0.0005, grad_fn=<SumBackward0>)

### 3. Influence Function Implementation

In [48]:
def upweighting_loss_influence_function(train_dataset, upweighted_data, upweighted_label, test_data, test_label, model, criterion):
    # Step 1. Compute the Inverse Hessian-vector product
    ihvp_eval = ihvp(train_dataset, test_data, test_label, model, criterion, 5000, 5)
    # Step 2. Compute the influence function
    first_grad = calc_first_order_gradient(criterion(model(upweighted_data), upweighted_label), model)
    influence = torch.dot(-ihvp_eval, first_grad)
    return influence

In [49]:
influence = upweighting_loss_influence_function(train_dataset, train_data[0], train_labels[0].view(1), x_test, y_test, base_model, base_model_criterion)

ihvp iteration 0 done and ihvp sum is 0.0005280785262584686
ihvp iteration 1 done and ihvp sum is 0.0005275940056890249
ihvp iteration 2 done and ihvp sum is 0.0005273295100778341
ihvp iteration 3 done and ihvp sum is 0.0005274936556816101
ihvp iteration 4 done and ihvp sum is 0.0005275439471006393


In [50]:
influence / n, calc_leave_out_loss_diff(base_model, initial_params, criterion_l2, x_test, y_test, t=10, leave_out_index=0)

loss_z_test_with_z: 0.002304940717294812, loss_z_test_without_z: 0.0023048899602144957, leave_out_loss_diff: -5.075708031654358e-08


(tensor(2.0989e-08, grad_fn=<DivBackward0>),
 tensor(-5.0757e-08, grad_fn=<SubBackward0>))